In [1]:
import json
import os
import re
from tqdm import tqdm

## clean fandom download page
there's lots of extra links which are definetly not benificial to our network analysis, as they might provide wrong links between nodes.
Therefore, here we plan to move the begining and the end part of the fandom page and just remain the middle part where tells the story of the character.



<font color=blue>(这边只是我尝试的部分！qinka如果整理只需要后面大标题处开始整理)

In [114]:
# try on one file
folder_path = 'D:\\PyCharm_project\\02805\\project\\data\\cha_fandom_page'
file_name = 'Caro_Snape.json'

path = os.path.join(folder_path,file_name)
with open(path,'r') as f:
    text = json.load(f)

In [115]:
text.keys()

dict_keys(['batchcomplete', 'warnings', 'query'])

In [116]:
text['warnings']

{'main': {'*': 'Subscribe to the mediawiki-api-announce mailing list at <https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce> for notice of API deprecations and breaking changes.'},
 'revisions': {'*': 'The parameter "rvparse" has been deprecated.'}}

In [118]:
text['query']

{'normalized': [{'from': 'Caro_Snape', 'to': 'Caro Snape'}],
 'pages': {'-1': {'ns': 0, 'title': 'Caro Snape', 'missing': ''}}}

In [117]:
id = list(text['query']['pages'].keys())[0]
print(id)

-1


In [74]:
raw_text = str(text['query']['pages'][id]['revisions'][0]['*'])
print(raw_text)

<div class="mw-parser-output"><table class="notice metadata" id="spoiler" style="text-align: center; background: beige; margin: 1em 05%; padding: 0 1em; border: #a00 1px solid;">

<tbody><tr>
<td><a href="https://static.wikia.nocookie.net/harrypotter/images/e/ea/Wizards_Unite_logo.jpg/revision/latest?cb=20181114170049" class="image"><img alt="Wizards Unite logo.jpg" src="https://static.wikia.nocookie.net/harrypotter/images/e/ea/Wizards_Unite_logo.jpg/revision/latest/scale-to-width-down/175?cb=20181114170049" decoding="async" width="175" height="98" data-image-name="Wizards Unite logo.jpg" data-image-key="Wizards_Unite_logo.jpg" /></a>
</td>
<td><b>Warning!</b>
<p>At least some content in this article is derived from information featured in <i><a href="/wiki/Harry_Potter:_Wizards_Unite" title="Harry Potter: Wizards Unite">Harry Potter: Wizards Unite</a></i>. As such, spoilers will be present within the article.
</p><p>For gameplay-specific information, please see the relevant article on

找呀找呀找规律  找到一些小规律：
1. start with '[source]'
2. end with 'Notes and references' (第一个出现的时候就掐掉)

In [81]:
text_without_structure = raw_text

# get the middle part where the text describe this person
get_middle_text_pattern = re.compile(r'\[Source\][\s\S]*Notes and references')
result= get_middle_text_pattern.findall(str(text_without_structure))[0]

In [82]:
print(result)

[Source]</a><span class="hidable-button"></span></small></nav>
</aside>

</p>
<div class="quote">
<dl><dd><span title="Source:&#160;Harry Potter and the Chamber of Secrets">"<i>Funny you should mention <a href="/wiki/Lucius_Malfoy" title="Lucius Malfoy">Draco's father</a>, [...] 'Let me show you the generous gift he's made to the Slytherin team.'</i>"</span></dd>
<dd>&#8212;Marcus admitting to accepting a bribe when determining a new <a href="/wiki/Seeker" title="Seeker">Seeker</a>.<sup class="noprint"><a href="/wiki/Harry_Potter_and_the_Chamber_of_Secrets" title="Harry Potter and the Chamber of Secrets">[src]</a></sup></dd>
<dd></dd></dl></div>
<p><b>Marcus Flint</b> (b. <a href="/wiki/1975" title="1975">1975</a>/<a href="/wiki/1976" title="1976">1976</a>)<sup id="cite_ref-DOB_1-1" class="reference"><a href="#cite_note-DOB-1">&#91;1&#93;</a></sup> was a <a href="/wiki/Wizardkind" title="Wizardkind">wizard</a> who attended <a href="/wiki/Hogwarts_School_of_Witchcraft_and_Wizardry" titl

In [80]:
# 但是这里我想先把所有'<>'框起来的玩意儿给去掉！ （这不适用于link提取因为我们需要这个东西但是！ 只是感觉这样子处理之后比较好看清楚结构）
# 以及之后的TF-IDF可以用到这个！
def get_clean_text(text):
    
    # remove all the html stuctures with '<   >'
    pattern=re.compile(r'<.*?>')
    out = re.sub(pattern, ' ', text)
    
    # remove all the blank and return symbols
    pattern_blank = re.compile(r'[\t ]+')
    out = re.sub(pattern_blank, ' ', out)
    
    # remove the '[ edit | edit source ]'
    pattern_edit = re.compile(r'\[ edit \| edit source \]')
    out = re.sub(pattern_edit, '', out)

    # remove the last and the first part
    pattern_last = re.compile(r'\[ edit[\s\S]*')
    out = re.sub(pattern_last, '', out)
    pattern_first = re.compile(r'\[Source\]')
    out = re.sub(pattern_first, '', out)
    
    return out

print(get_clean_text(result))

 
 

 
 
 " Funny you should mention Draco's father , [...] 'Let me show you the generous gift he's made to the Slytherin team.' " 
 &#8212;Marcus admitting to accepting a bribe when determining a new Seeker . [src] 
 
 Marcus Flint (b. 1975 / 1976 ) &#91;1&#93; was a wizard who attended Hogwarts School of Witchcraft and Wizardry from 1987 to 1994 and was sorted into Slytherin house. He was a Chaser on the Slytherin Quidditch team , as well as its Captain from at least 1991 to 1994 . 
 
 Contents 
 
 1 Biography 
 
 1.1 Early life 
 1.2 Hogwarts years (1987-1994) 
 
 1.2.1 Early years 
 1.2.2 Sixth year 
 1.2.3 Seventh year 
 
 
 1.3 Later Life 
 
 
 2 Physical appearance 
 3 Personality and traits 
 4 Magical abilities and skills 
 5 Etymology 
 6 Behind the scenes 
 7 Appearances 
 8 Notes and references 
 
 

 Biography  
 Early life  
 Marcus was born in September 1975 to August 1976 &#91;1&#93; in Great Britain . He was a descendant of the pure-blood Flint family and perhaps relat

### work with all fandom pages (前面只是我尝试的部分 从这边开始是对全部文件的操作！qinka如果整理只需要从这里整理)

In [110]:
ori_folder_path = './data/cha_fandom_page'
new_folder_path = './data/cha_fandom_page_clean'
text_folder_path = './data/cha_fandom_page_text'

if not os.path.exists(new_folder_path):
    os.mkdir(new_folder_path)
if not os.path.exists(text_folder_path):
    os.mkdir(text_folder_path)  

In [111]:
# define the function for cleanning the text from page_clean to page_text
def get_clean_text(text):
    
    # remove all the html stuctures with '<   >'
    pattern=re.compile(r'<[\s\S]*?>')
    out = re.sub(pattern, ' ', text)
    
    # remove all the blank and \t symbols
    pattern_blank = re.compile(r'[\t ]+')
    out = re.sub(pattern_blank, ' ', out)
    
#     # replace multiple '\n' into one '\n'
#     pattern_mul_return = re.compile(r'\n[\n ]+')
#     out = re.sub(pattern_mul_return, '\n', out)
    
    # remove the '[ edit | edit source ]'
    pattern_edit = re.compile(r'\[ edit \| edit source \]')
    out = re.sub(pattern_edit, '', out)

    # remove the last and the first part
    pattern_last = re.compile(r'\[ edit[\s\S]*')
    out = re.sub(pattern_last, '', out)
    pattern_first = re.compile(r'\[Source\]')
    out = re.sub(pattern_first, '', out)
    
    return out

In [121]:
# from ori folder get each file and process 

file_list = os.listdir(ori_folder_path)
for i,file in enumerate(file_list[580:]):
    print('{}\tStart processing file {}...'.format(i+580,file),end=' ')
    file_path = os.path.join(ori_folder_path,file)
    with open(file_path,'r') as f:
        text = json.load(f)
    
    str_id = str(list(text['query']['pages'].keys())[0])
    if str_id == '-1':
        print('***WARNING ON: {} - no original page information'.format(file))
        continue
        
    raw_text = str(text['query']['pages'][str_id]['revisions'][0]['*'])
    
    # get the middle part where the text describe this person
#     get_middle_text_pattern = re.compile(r'\[Source\][\s\S]*Notes and references')
    self_name = file.split('.json')[0]
#     match_re = self_name+'[\s\S]*Notes and references'
#     print(match_re)
    get_middle_text_pattern = re.compile(r'[\s\S]*Notes and references')
    results= get_middle_text_pattern.findall(raw_text)
    
    
    if len(results) == 0:
        print('***WARNING ON: {} - no enough length for results'.format(file))
        continue
    
    else:
        result = results[0]
    
    # store this text into txt for link extraction
#     result = result.decode('utf-8')
    with open(os.path.join(new_folder_path,self_name+'.txt'),'w',encoding='utf-8') as f:
        f.write(result) 
    
    # clean all the html structures for later on nlp analysis
    
    only_text = get_clean_text(result)
    # store this text into txt for nlp analysis
    with open(os.path.join(text_folder_path,self_name+'.txt'),'w',encoding='utf-8') as f:
        f.write(only_text) 
    
    print('DONE.')
    
    

580	Start processing file Caro_Snape.json... ***WARNING ON: Caro_Snape.json - no original page information
581	Start processing file Carrot_Wand.json... DONE.
582	Start processing file Carrow_(1920s).json... DONE.
583	Start processing file Caspar_Crouch's_daughters.json... DONE.
584	Start processing file Caspar_Crouch's_son.json... DONE.
585	Start processing file Caspar_Crouch.json... DONE.
586	Start processing file Cassandra's_uncle.json... DONE.
587	Start processing file Cassandra_(Hogwarts_student).json... DONE.
588	Start processing file Cassandra_Trelawney.json... DONE.
589	Start processing file Cassandra_Vablatsky.json... DONE.
590	Start processing file Cassiopeia_Black.json... DONE.
591	Start processing file Cassius_Bell.json... DONE.
592	Start processing file Cassius_Warrington.json... DONE.
593	Start processing file Cast-a-Spell_training_room.json... DONE.
594	Start processing file Catchpole.json... DONE.
595	Start processing file Catriona_McCormack.json... DONE.
596	Start proc

721	Start processing file Corban_Yaxley.json... DONE.
722	Start processing file Cordelia_Gifford.json... DONE.
723	Start processing file Cordelia_Misericordia.json... DONE.
724	Start processing file Cordelia_Vexmoor.json... DONE.
725	Start processing file Cormack_McLeod.json... DONE.
726	Start processing file Cormac_McLaggen's_father.json... DONE.
727	Start processing file Cormac_McLaggen.json... DONE.
728	Start processing file Cormac_O'Brien.json... DONE.
729	Start processing file Cornelius_Agrippa.json... DONE.
730	Start processing file Cornelius_Fudge's_wand.json... DONE.
731	Start processing file Cornelius_Fudge's_wife.json... DONE.
732	Start processing file Cornelius_Fudge.json... DONE.
733	Start processing file Corrine_Cribb.json... DONE.
734	Start processing file Corvinus_Gaunt.json... DONE.
735	Start processing file Corvus_Lestrange_(I).json... DONE.
736	Start processing file Corvus_Lestrange_(II).json... DONE.
737	Start processing file Corvus_Lestrange_(III).json... DONE.
738	

862	Start processing file Delphini.json... DONE.
863	Start processing file Demelza_Robins.json... DONE.
864	Start processing file Dementor.json... DONE.
865	Start processing file Dementor_who_searched_Harry_Potter's_compartment.json... DONE.
866	Start processing file Dempster_Wiggleswade.json... DONE.
867	Start processing file Denison_Frisby.json... DONE.
868	Start processing file Dennis.json... DONE.
869	Start processing file Dennis_(toad).json... DONE.
870	Start processing file Dennis_Aldermaston's_mother.json... DONE.
871	Start processing file Dennis_Aldermaston.json... DONE.
872	Start processing file Dennis_Bishop.json... DONE.
873	Start processing file Dennis_Creevey.json... DONE.
874	Start processing file Dennis_Moon.json... DONE.
875	Start processing file Dentist.json... DONE.
876	Start processing file Derek.json... DONE.
877	Start processing file Derek_(turtle_talker).json... DONE.
878	Start processing file Derek_(Wizard_Card_Collectors'_Club).json... DONE.
879	Start processing

1142	Start processing file Ethelred_the_Ever-Ready.json... DONE.
1143	Start processing file Euan_Abercrombie.json... DONE.
1144	Start processing file Eugene_Belcher.json... DONE.
1145	Start processing file Eugene_McAllister.json... DONE.
1146	Start processing file Eugene_Warmsley.json... DONE.
1147	Start processing file Eugenia_Jenkins.json... DONE.
1148	Start processing file Eulalie_Hicks.json... DONE.
1149	Start processing file Eunice_Cabanart.json... DONE.
1150	Start processing file Eunice_Murray.json... DONE.
1151	Start processing file Euphemia_Potter.json... DONE.
1152	Start processing file Euphemia_Rowle.json... DONE.
1153	Start processing file Eupraxia_Mole.json... DONE.
1154	Start processing file Eurig_Cadwallader.json... DONE.
1155	Start processing file Eustace_Fawley's_father.json... DONE.
1156	Start processing file Eustace_Fawley.json... DONE.
1157	Start processing file Evaan_Mathew.json... DONE.
1158	Start processing file Evangeline_Orpington's_wand.json... ***WARNING ON: E

1280	Start processing file Fred_and_George_Weasley.json... ***WARNING ON: Fred_and_George_Weasley.json - no enough length for results
1281	Start processing file Fred_Baille.json... DONE.
1282	Start processing file Fred_Weasley's_wand.json... DONE.
1283	Start processing file Fred_Weasley.json... DONE.
1284	Start processing file Fred_Weasley_II.json... DONE.
1285	Start processing file Friar.json... DONE.
1286	Start processing file Fridwulfa.json... DONE.
1287	Start processing file Fubster.json... DONE.
1288	Start processing file Fulbert_the_Fearful.json... DONE.
1289	Start processing file Fulcran_Lestrange_(I).json... DONE.
1290	Start processing file Fulcran_Lestrange_(II).json... DONE.
1291	Start processing file Fulcran_Lestrange_(III).json... DONE.
1292	Start processing file Fuzzclaw.json... DONE.
1293	Start processing file G._F.'s_grandfather.json... DONE.
1294	Start processing file G._F..json... DONE.
1295	Start processing file G._Forbes.json... DONE.
1296	Start processing file G8_3$

...

3359	Start processing file Smoky-eyed_Potions_professor.json... DONE.
3360	Start processing file Snaglok.json... DONE.
3361	Start processing file Snake.json... DONE.
3362	Start processing file Snargaluff.json... DONE.
3363	Start processing file Snatcher_individual_infobox.json... ***WARNING ON: Snatcher_individual_infobox.json - no original page information
3364	Start processing file Snickerton.json... DONE.
3365	Start processing file Snowy.json... DONE.
3366	Start processing file Society_for_the_Support_of_Squibs.json... DONE.
3367	Start processing file Sofie.json... DONE.
3368	Start processing file Sol_Campbell.json... DONE.
3369	Start processing file Sonnleitner.json... DONE.
3370	Start processing file Sophia_Prickett.json... DONE.
3371	Start processing file Sophie_Berger.json... DONE.
3372	Start processing file Sophie_Duchamps.json... DONE.
3373	Start processing file Sophie_Roper.json... DONE.
3374	Start processing file Sorting_Hat.json... DONE.
3375	Start processing file Sorting_s

3497	Start processing file Thaddeus_Thurkell.json... DONE.
3498	Start processing file Thelma_Holmes.json... DONE.
3499	Start processing file Theodard_Fontaine.json... DONE.
3500	Start processing file Theodore_Nott's_mother.json... DONE.
3501	Start processing file Theodore_Nott.json... DONE.
3502	Start processing file Theophilus_Abbot.json... DONE.
3503	Start processing file Theoretician.json... DONE.
3504	Start processing file Theo_Strongarm.json... DONE.
3505	Start processing file Theresa_Stadt.json... DONE.
3506	Start processing file Theseus_Scamander's_wand.json... DONE.
3507	Start processing file Theseus_Scamander.json... DONE.
3508	Start processing file Thestral.json... DONE.
3509	Start processing file The_Blind_Pig_Poster.json... DONE.
3510	Start processing file The_Daily_Prophet's_Bestseller_List.json... DONE.
3511	Start processing file The_Daily_Prophet_Competition.json... DONE.
3512	Start processing file The_Daily_Prophet_Press.json... DONE.
3513	Start processing file The_Fate

3737	Start processing file Unidentified_five_Animagi.json... DONE.
3738	Start processing file Unidentified_five_Muggles.json... DONE.
3739	Start processing file Unidentified_five_Muggles_who_saw_the_Flying_Ford_Anglia.json... DONE.
3740	Start processing file Unidentified_florist_in_Diagon_Alley.json... DONE.
3741	Start processing file Unidentified_fourth_year_Chaser.json... DONE.
3742	Start processing file Unidentified_fourth_year_student.json... DONE.
3743	Start processing file Unidentified_French_Ministry_of_Magic_receptionist.json... DONE.
3744	Start processing file Unidentified_French_policeman.json... DONE.
3745	Start processing file Unidentified_friend_of_McClaggan.json... DONE.
3746	Start processing file Unidentified_frizzy-haired_Wizengamot_member.json... DONE.
3747	Start processing file Unidentified_frowning_Ministry_of_Magic_wizard_who_murmured_instructions_to_a_quill.json... DONE.
3748	Start processing file Unidentified_Ghosts.json... DONE.
3749	Start processing file Unident

3941	Start processing file Unidentified_Ravenclaw_black_girl_during_the_Battle_of_Hogwarts.json... DONE.
3942	Start processing file Unidentified_Ravenclaw_boy.json... DONE.
3943	Start processing file Unidentified_Ravenclaw_boy_(II).json... DONE.
3944	Start processing file Unidentified_Ravenclaw_boy_at_Hogwarts.json... DONE.
3945	Start processing file Unidentified_Ravenclaw_fourth_years.json... DONE.
3946	Start processing file Unidentified_Ravenclaw_Girl_during_the_Battle_of_Hogwarts.json... DONE.
3947	Start processing file Unidentified_Ravenclaw_lady.json... DONE.
3948	Start processing file Unidentified_Ravenclaw_member_of_Dumbledore's_Army.json... DONE.
3949	Start processing file Unidentified_Ravenclaw_prefect.json... DONE.
3950	Start processing file Unidentified_Ravenclaw_prefect_(II).json... DONE.
3951	Start processing file Unidentified_Ravenclaw_Seeker.json... DONE.
3952	Start processing file Unidentified_Ravenclaw_student_asked_to_the_Yule_Ball_by_a_Durmstrang_student.json... DONE

4189	Start processing file Winky.json... DONE.
4190	Start processing file Winky_Crockett_(Gryffindor).json... DONE.
4191	Start processing file Winky_Crockett_(Slytherin).json... DONE.
4192	Start processing file Winogrand.json... DONE.
4193	Start processing file Winstanley.json... DONE.
4194	Start processing file Winston's_niece.json... DONE.
4195	Start processing file Winston_Churchill.json... DONE.
4196	Start processing file Wiseacre.json... DONE.
4197	Start processing file Witch_and_Wizard_Couple.json... DONE.
4198	Start processing file Witch_with_a_hairy_chin.json... DONE.
4199	Start processing file Wizard's_Chess.json... DONE.
4200	Start processing file Wizard's_father.json... DONE.
4201	Start processing file Wizarding_Wireless_Network_News.json... DONE.
4202	Start processing file Wizard_(The_Wizard_and_the_Hopping_Pot).json... DONE.
4203	Start processing file Wizard_in_the_Leaky_Cauldron.json... DONE.
4204	Start processing file Wizard_in_white_robes.json... DONE.
4205	Start proces

#### GREAT JOB! We have clean all the pages and store the files in ./data/cha_fandom_page_clean (for extract links) and ./data/cha_fandom_page_text (for nlp analysis)

In [2]:
# get the link again from the clean file
node_edges = {}
folder = './data/cha_fandom_page_clean/'

In [3]:
# get all the notes
all_nodes = []
all_cha_json_files = os.listdir(folder)
for each in all_cha_json_files:
    cha_name = each.split('.txt')[0]
    all_nodes.append(cha_name)

num_of_nodes = len(all_nodes)
print('the number of nodes: {}'.format(num_of_nodes))

the number of nodes: 4215


In [4]:
# define the function for extract links from 
def get_links(text):
    pattern=re.compile(r'<a href="/wiki/.*?" title=".*?">')
    result= pattern.findall(str(text))
    list_link=[]
    for x in result:
        x=x.split(' title')
        href=x[0].split('href="/wiki/')[-1].split('"')[0]
        list_link.append(href)
    return list_link

In [5]:
for i,node in enumerate(all_nodes):
    # 2.1 get the json file 
    txt_path = os.path.join(folder,node+'.txt')
    with open(txt_path,'r',encoding='utf-8') as f:
        text = f.read()

#     print(str(text)[:30])
    
    # get the links from text
    raw_links = get_links(text)
    
    # avoid repeating
    raw_links = list(set(raw_links))
    
    # check if links are from the nodes_list
    true_links = []
    for link in raw_links:
        if link in all_nodes:
            true_links.append(link)
    
    node_edges[node] = true_links
    print('{}/{}\tnode {} has {} links'.format(i+1,num_of_nodes,node,len(true_links)))
    print('{}\n'.format(true_links))

1/4215	node 16th_century_Potions_Professor has 2 links
['Professor', 'Zygmunt_Budge']

2/4215	node 1992_Gryffindor-Slytherin_Quidditch_match_spectators has 12 links
['Severus_Snape', 'Filius_Flitwick', 'Hermione_Granger', 'Irma_Pince', 'Lucius_Malfoy', 'Pomona_Sprout', 'Draco_Malfoy', 'Albus_Dumbledore', 'Harry_Potter', 'Lee_Jordan', 'Minerva_McGonagall', 'Gilderoy_Lockhart']

3/4215	node 1996_Gryffindor_Quidditch_Keeper_trials_spectators has 4 links
['Cormac_McLaggen', 'Hermione_Granger', 'Harry_Potter', 'Ronald_Weasley']

4/4215	node 2-Headed_Baby has 0 links
[]

5/4215	node A.J._Mennett has 0 links
[]

6/4215	node A._Criddle has 0 links
[]

7/4215	node A._Fenetre has 2 links
['Cornelius_Fudge', 'Sirius_Black']

8/4215	node A._Osborn has 1 links
['Non-magic_people']

9/4215	node Aamir_Loonat has 0 links
[]

10/4215	node Abbott has 6 links
['Giffard_Abbott', 'Half-blood', 'Muggle-born', 'Non-magic_people', 'Pure-blood', 'Hannah_Abbott']

11/4215	node Abel_Treetops has 1 links
['Wampus

88/4215	node Albus_Dumbledore has 221 links
['Herbert_Beery', 'Phineas_Nigellus_Black', 'Bertha_Jorkins', 'Susan_Bones', 'Bartemius_Crouch_Junior', 'Cuthbert_Binns', 'Corvus_Lestrange_(V)', 'Vernon_Dursley', 'Davet_Dumont', 'Honoria', 'Bob_Ogden', 'Curse-Breaker', 'Charity_Burbage', 'Donaghan_Tremlett', 'Frank', 'Edgar_Bones', 'Alice_Longbottom', 'Arthur_Weasley', 'Cornelius_Fudge', 'Forbidden_Forest_Centaur_colony', 'Porpentina_Goldstein', 'Bartemius_Crouch_Senior', 'Albus_Potter', 'Fawkes', 'Sword_of_Gryffindor', 'Arabella_Figg', 'Headmaster_portraits', 'Filius_Flitwick', 'Fenrir_Greyback', 'Alastor_Moody', 'Buckbeak', 'Frank_Longbottom', 'Hope_Howell', 'Chessboard_Chamber', 'Dorcas_Meadowes', 'Poppy_Pomfrey', 'Lyall_Lupin', 'Morfin_Gaunt', 'Hannah_Abbott', 'Fred_Weasley', 'Igor_Karkaroff', 'Secret_Keeper', 'Horcrux', 'Michael_Corner', 'Tom_Riddle', 'Acromantula', 'Remus_Lupin', 'Sybill_Trelawney', 'James_Potter_I', 'Gilderoy_Lockhart', 'Narcissa_Malfoy', 'Edward_Tonks', 'Duncan_Ashe

130/4215	node Alphard_Black has 12 links
['Pollux_Black', 'Walburga_Black', 'Regulus_Black', 'Irma_Crabbe', 'Andromeda_Tonks', 'Sirius_Black', 'Bellatrix_Lestrange', 'Druella_Black', 'Pure-blood', 'Narcissa_Malfoy', 'Cygnus_Black_III', 'Orion_Black']

131/4215	node Altheda's_wand has 3 links
['Evil_sorcerer', 'Beedle_the_Bard', 'Altheda']

132/4215	node Altheda has 7 links
['Potioneer', 'Wand', 'Beedle_the_Bard', 'Evil_sorcerer', 'Amata', 'Asha', 'Sir_Luckless']

133/4215	node Alvin_Simmons has 0 links
[]

134/4215	node Alys has 2 links
['Professor', 'Horace_Slughorn']

135/4215	node Amanda has 5 links
['Filius_Flitwick', 'Professor', 'Rolanda_Hooch', 'Sorting_Hat', 'Mandy_Brocklehurst']

136/4215	node Amanda_Pettet has 2 links
['Robert_Bliss', 'Non-magic_people']

137/4215	node Amarillo_Lestoat has 0 links
[]

138/4215	node Amata's_lover has 2 links
['Amata', 'Beedle_the_Bard']

139/4215	node Amata has 7 links
['Healer', 'Wand', 'Beedle_the_Bard', 'Non-magic_people', 'Asha', 'Sir_Luck

209/4215	node Antioch_Peverell's_killer has 4 links
['Mykew_Gregorovitch', 'Antioch_Peverell', 'Wand', 'Death_(The_Tale_of_the_Three_Brothers)']

210/4215	node Antioch_Peverell has 10 links
['Albus_Dumbledore', 'Tom_Riddle', 'Wandmaker', 'Wand', 'Beedle_the_Bard', 'Harry_Potter', 'Cadmus_Peverell', 'Pure-blood', 'Death_(The_Tale_of_the_Three_Brothers)', 'Ignotus_Peverell']

211/4215	node Antonia_Creaseworthy has 3 links
['Knight', 'Sybill_Trelawney', 'Portrait']

212/4215	node Antonin_Dolohov has 34 links
['Blood_traitor', 'Molly_Weasley', 'Gideon_Prewett', 'Dean_Thomas', 'Pius_Thicknesse', 'Nagini', 'Neville_Longbottom', 'Parvati_Patil', 'Charity_Burbage', 'Harry_Potter', 'Sirius_Black', 'Bellatrix_Lestrange', 'Non-magic_people', 'Pure-blood', 'Jugson', 'Filius_Flitwick', 'Alastor_Moody', 'Ronald_Weasley', 'Half-blood', 'Nymphadora_Tonks', 'Lucius_Malfoy', 'Muggle-born', 'Draco_Malfoy', 'Albus_Dumbledore', 'Thorfinn_Rowle', 'Severus_Snape', 'Igor_Karkaroff', 'Tom_Riddle', 'Hermione_Gr

271/4215	node Arthur_Weasley has 86 links
['Edward_Lupin', 'Blood_traitor', 'Dominique_Weasley', 'Garrick_Ollivander', 'Fleur_Delacour', 'Cornelius_Fudge', 'Calamity_Investigator', 'Bartemius_Crouch_Senior', 'Audrey_Weasley', 'Albus_Potter', 'George_Weasley', 'Roxanne_Weasley', 'Fenrir_Greyback', 'Alastor_Moody', 'Bilius', 'Muggle-born', 'Half-breed', 'Fred_Weasley', 'Horcrux', 'Tom_Riddle', 'Remus_Lupin', 'Gilderoy_Lockhart', 'Molly_Weasley', 'Gideon_Prewett', 'Callidora_Black', 'Wand', 'Molly_Weasley_II', 'Dirk_Cresswell', 'Winky', 'Septimus_Weasley', 'Kingsley_Shacklebolt', 'Dolores_Umbridge', 'Half-blood', 'Nymphadora_Tonks', 'Victoire_Weasley', 'Draco_Malfoy', 'Augustus_Pye', 'Albus_Dumbledore', 'Harfang_Longbottom', 'Louis_Weasley', 'Ludovic_Bagman', 'Peter_Pettigrew', 'Severus_Snape', 'Albert_Runcorn', 'Rubeus_Hagrid', 'Dark_Mark', 'Charis_Black', 'Pius_Thicknesse', 'Cedrella_Black', 'Nagini', 'Rita_Skeeter', 'Rufus_Scrimgeour', 'Charles_Weasley', 'Arcturus_Black_II', 'Fred_Weas

...

3862/4215	node Unidentified_nine-year_old has 0 links
[]

3863/4215	node Unidentified_Non-magique_Father has 3 links
['Unidentified_Non-magique_Mother', 'Credence_Barebone', 'Non-magic_people']

3864/4215	node Unidentified_Non-magique_Mother has 3 links
['Credence_Barebone', 'Non-magic_people', 'Unidentified_Non-magique_Father']

3865/4215	node Unidentified_Non-magique_toddler has 5 links
['Unidentified_Non-magique_Mother', 'Credence_Barebone', 'Non-magic_people', 'Carrow_(1920s)', 'Unidentified_Non-magique_Father']

3866/4215	node Unidentified_nose-picking_wizard has 1 links
['Ronald_Weasley']

3867/4215	node Unidentified_Obliviator has 0 links
[]

3868/4215	node Unidentified_Old_Man has 3 links
['Beedle_the_Bard', 'Wizard_(The_Wizard_and_the_Hopping_Pot)', 'Non-magic_people']

3869/4215	node Unidentified_old_tramp has 7 links
['Muggle_tramp', 'Petunia_Dursley', 'Vernon_Dursley', 'Harry_Potter', 'Bartemius_Crouch_Senior', 'Dudley_Dursley', 'Non-magic_people']

3870/4215	node Unidentif

3941/4215	node Unidentified_son has 6 links
['Unidentified_young_husband', 'Unidentified_grandfather', 'Unidentified_daughter', 'Non-magic_people', 'Unidentified_grandmother', 'Unidentified_young_wife']

3942/4215	node Unidentified_spellbook has 4 links
['Ronald_Weasley', 'Dolores_Umbridge', 'Hermione_Granger', 'Harry_Potter']

3943/4215	node Unidentified_studying_girls has 4 links
['First_Headmaster_of_Hogwarts', 'Percy_Weasley', 'Nicholas_de_Mimsy-Porpington', 'Penelope_Clearwater']

3944/4215	node Unidentified_St_Mungo's_girl_patient's_father has 0 links
[]

3945/4215	node Unidentified_St_Mungo's_patient's_brother has 0 links
[]

3946/4215	node Unidentified_St_Mungo's_patient has 0 links
[]

3947/4215	node Unidentified_Sudanese_Obscurial has 4 links
['Newton_Scamander', 'Obscurial', 'Magizoologist', 'Non-magic_people']

3948/4215	node Unidentified_teacher_at_St._Grogory's_Primary_School has 2 links
['Harry_Potter', 'Non-magic_people']

3949/4215	node Unidentified_ten-year-old has 3 

4020/4215	node Vincent_Davis has 0 links
[]

4021/4215	node Vincent_Duc_de_Trefle-Picques has 0 links
[]

4022/4215	node Vinda_Rosier has 17 links
['Felix_Rosier', 'Evan_Rosier', 'Abernathy', 'Tom_Riddle', 'Half-blood', 'Rosier_(male)', 'Porpentina_Goldstein', 'Gellert_Grindelwald', 'Minette_Lestrange', 'Queenie_Goldstein', 'Leta_Lestrange', 'Bellatrix_Lestrange', 'Non-magic_people', 'Druella_Black', 'Pure-blood', 'Alliance', 'Carrow_(1920s)']

4023/4215	node Vindictus_Viridian has 6 links
['Professor', 'Hermione_Granger', 'Potioneer', 'Harry_Potter', 'Myrtle_Warren', 'Headmaster_portraits']

4024/4215	node Viola_Landers has 0 links
[]

4025/4215	node Viola_Richmond has 0 links
[]

4026/4215	node Violet-cloaked_wizard has 9 links
['Filius_Flitwick', 'Dedalus_Diggle', 'Tom_Riddle', 'Rubeus_Hagrid', 'Vernon_Dursley', 'Harry_Potter', 'Albus_Dumbledore', 'Non-magic_people', 'Minerva_McGonagall']

4027/4215	node Violet has 5 links
['William_Weasley', 'Harry_Potter', 'Violet_Tillyman', 'Fat_

4095/4215	node Webster_Boot has 18 links
['James_Steward', 'Wampus_cat', 'Rionach_Steward', 'Chadwick_Boot', 'Half-blood', 'Gormlaith_Gaunt', 'Pukwudgie', 'Horned_Serpent', 'Auror', 'Parseltongue', 'Wand', 'Martha_Steward_II', 'William_(Pukwudgie)', 'Terry_Boot', 'Pure-blood', 'William_Sayre', 'Josefina_Calderon', 'Isolt_Sayre']

4096/4215	node Wee_Willykins has 2 links
['Beatrix_Bloxam', 'Beedle_the_Bard']

4097/4215	node Wendelin_the_Weird has 4 links
['Bathilda_Bagshot', 'Harry_Potter', 'Non-magic_people', 'Portrait']

4098/4215	node Wendy_Slinkhard has 2 links
['Horace_Slughorn', 'Wilbert_Slinkhard']

4099/4215	node Werewolf has 45 links
['Montgomery', 'Edward_Lupin', 'Anonymous_author_of_Hairy_Snout,_Human_Heart', 'Damocles_Belby', 'Professor', 'Werewolf_army', 'Lavender_Brown', 'Cecil_Lee', 'Penny_Haywood', 'Unicorn', 'Silas_Crump', 'Newton_Scamander', 'Arthur_Weasley', 'Quirinus_Quirrell', 'Scarlett', 'Harry_Potter', 'Marlowe_Forfang', 'Bellatrix_Lestrange', 'Non-magic_people', 

4172/4215	node Y._Blishen has 2 links
['F._Wakefield', 'J._Ward']

4173/4215	node Yann_Fredericks has 9 links
['Rolanda_Hooch', 'Tom_Riddle', 'Karl_Jenkins', 'Hermione_Granger', 'Muggle-born', 'Scorpius_Malfoy', 'Albus_Potter', 'Harry_Potter', 'Rose_Granger-Weasley']

4174/4215	node Yardley_Platt has 0 links
[]

4175/4215	node Yatin_Bhagat has 0 links
[]

4176/4215	node Yaxley_(1920s) has 4 links
['Leta_Lestrange', 'Theseus_Scamander', 'Pure-blood', 'Hector_Fawley']

4177/4215	node Ya_Zhou has 1 links
['Henry_Shaw_Junior']

4178/4215	node Yew_and_unicorn_hair_wand has 4 links
['Tom_Riddle', 'Wand', 'Garrick_Ollivander', 'Harry_Potter']

4179/4215	node Ye_Olde_Curiosity_Shop_shopkeeper has 1 links
['Harry_Potter']

4180/4215	node Yolanda_Reedham has 0 links
[]

4181/4215	node Yoshihiro_Suzuki has 2 links
['Natsumi_Suzuki', 'Lucy_Hurst']

4182/4215	node Yoshi_Wakahisa has 0 links
[]

4183/4215	node Young_witch_maid has 1 links
['Tom']

4184/4215	node Young_woman's_child has 4 links
['You

In [130]:
json_file = './data/nodes_egdes_new.json'
with open(json_file,'w') as f:
    json.dump(node_edges,f)